In [1]:
import os
import requests
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool


from langgraph.checkpoint.memory import MemorySaver


from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode



OPENWEATHER_API_KEY = "1d6b1a0c90d3d95094fa968ac8fdd24e"
OPENWEATHER_URL = "http://api.openweathermap.org/data/2.5/weather"


def fetch_weather(location: str):
    try:
        params = {
            "q": location,
            "appid": OPENWEATHER_API_KEY,
            "units": "metric",
        }

        response = requests.get(OPENWEATHER_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            weather = data["weather"][0]["description"]
            temp = data["main"]["temp"]
            return f"The weather in {location} is {weather} with a temperature of {temp}°C."
        elif response.status_code == 404:
            return f"Could not find weather information for {location}."
        else:
            return "Error fetching weather data. Please try again later."
    except Exception as e:
        return f"An error occurred: {str(e)}"



@tool

def search(query: str):
    """Search for weather information based on the query."""
    words = query.lower().split()
    if "weather" in words:
        try:
            location_index = words.index("in") + 1


            location = " ".join(words[location_index:])
            return fetch_weather(location)
        except (ValueError, IndexError):
            return "I couldn't understand the location. Please try rephrasing."
    return "I can only provide weather information."


tools = [search]



tool_node = ToolNode(tools)


model = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    api_key=os.getenv("CLAUDE_API_KEY"),
    temperature=0,
).bind_tools(tools)



def should_continue(state: MessagesState) -> Literal["tools", END]:  # type: ignore
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"


    return END



def call_model(state: MessagesState):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}



workflow = StateGraph(MessagesState)



workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)



workflow.add_edge(START, "agent")



workflow.add_conditional_edges(
    "agent",
    should_continue,
)



workflow.add_edge("tools", "agent")



checkpointer = MemorySaver()



app = workflow.compile(checkpointer=checkpointer)



user_input = "What's the weather in Dhaka?"
final_state = app.invoke(

    {"messages": [HumanMessage(content=user_input)]},

    config={"configurable": {"thread_id": 42}},
)



print(final_state["messages"][-1].content)

Based on the search results, I can provide you with the current weather information for Dhaka:

The weather in Dhaka is currently hazy with a temperature of 23.99°C (about 75°F). 

Haze typically indicates that there's some level of atmospheric pollution or fine dust particles in the air, which can reduce visibility. It's a relatively mild temperature, which is comfortable for most people.

Is there anything else you'd like to know about the weather in Dhaka or any other location?
